# Generación de actividades de forma sintética

In [1]:
import pandas as pd
from faker import Faker
import random
import time
import requests
import folium

Faker.seed(42)
fake = Faker('es_MX')
random.seed(42)

In [2]:
df_destinos = pd.read_csv('../merging-hospedajes/destinos.csv')

## Establecer las actividades y su categoria

In [3]:
tipos_actividad = [
    # Cultura y arte
    'Recorrido histórico por el centro',
    'Tour de arte urbano / grafitis',
    'Visita guiada a museos',
    'Taller de cerámica o pintura local',
    'Noche de teatro local',
    'Exposición de arte contemporáneo',
    'Clase de baile típico (tango, flamenco, samba…)',
    # Gastronomía
    'Tour gastronómico callejero',
    'Cata de vinos o cervezas artesanales',
    'Cena con familia local (experiencia inmersiva)',
    'Clase de cocina tradicional',
    'Ruta de tapas o comida típica',
    'Visita a mercados populares',
    # Naturaleza y aventura
    'Caminata por reservas naturales',
    'Avistamiento de aves o fauna local',
    'Paseo en bicicleta por parques o miradores',
    'Kayak o paddle en lagos o ríos',
    'Senderismo guiado en montañas cercanas',
    'Excursión a cascadas o playas',
    # Fotografía y experiencias únicas
    'Sesión fotográfica con trajes típicos',
    'Tour en dron (filmación de video personalizado)',
    'Experiencia de realidad virtual sobre la ciudad',
    # Eventos y festivales
    'Festival de música local / conciertos al aire libre',
    'Fiesta tradicional / desfile cultural',
    'Feria artesanal',
    'Proyección de cine al aire libre',
    'Festival gastronómico',
    'Celebración religiosa o patronal',
    # Historia y religión
    'Tour de iglesias y arquitectura colonial',
    'Ruta de mitos y leyendas locales',
    'Visita a templos, sinagogas o mezquitas',
    'Cementerios históricos y relatos',
]

categorias = [
'Tour',
'Excursión',
'Visita Guiada',
'Aventura',
'Gastronomía',
'Cultural',
'Deportiva',
'Educativa',
'Recreativa',
'Religiosa',
'Ecoturismo',
'Festival',
'Concierto',
'Exposición',
'Taller',
'Conferencia',
'Feria',
'Mercado',
'Playa',
'Parque Temático',
]


## Generar actividades

In [4]:
min_actividades_por_destino = 50
max_actividades_por_destino = 150
min_precio = 50
max_precio = 5000

actividades = list()
direcciones = list()

actividad_id = 1
# Para evitar conflictos con direcciones de hospedajes (de momento hay ~20k hospedajes)
direcciones_id = 100000

def generar_direccion_para_actividad(destino):
    global direcciones_id
    direccion_id = direcciones_id

    direccion = dict()
    direccion['nombre'] = fake.street_name()
    direccion['estado'] = destino.estado
    direccion['municipio'] = destino.municipio
    direccion['ciudad'] = destino.ciudad
    direccion['cp'] = destino.codigopostal
    lat = random.uniform(destino.min_lat, destino.max_lat)
    lon = random.uniform(destino.min_lon, destino.max_lon)
    direccion['lat'] = lat
    direccion['lon'] = lon
    direccion['id'] = direccion_id
    direcciones.append(direccion)

    direcciones_id += 1
    return direccion_id

def generar_actividades():
    global actividad_id
    for index, destino in df_destinos.iterrows():
        cant_actividades = random.randint(min_actividades_por_destino, max_actividades_por_destino)

        for i in range(cant_actividades):
            actividad = dict()
            categoria = random.choice(categorias)
            actividad['destino_id'] = index + 1
            actividad['direccion_id'] = generar_direccion_para_actividad(destino)
            actividad['nombre'] = categoria + ' ' + fake.word()
            actividad['descripcion'] = fake.text(100)
            actividad['precio'] = round(random.uniform(min_precio, max_precio), 2)
            actividad['categoria'] = categoria
            actividad['id'] = actividad_id

            actividad_id += 1
            actividades.append(actividad)

In [5]:
generar_actividades()

In [6]:
df_actividades = pd.DataFrame(actividades)
df_direcciones = pd.DataFrame(direcciones)

In [7]:
df_actividades

,destino_id,direccion_id,nombre,descripcion,precio,categoria,id
0,1,100000,Aventura exercitationem,Qui perspiciatis tempora assumenda. Adipisci a...,1154.89,Aventura,1
1,1,100001,Aventura repellat,Aliquid dolorem enim aspernatur. Sint fugit is...,480.35,Aventura,2
2,1,100002,Exposición ipsam,Cum perspiciatis nisi harum nisi iusto dolorum...,1201.67,Exposición,3
3,1,100003,Parque Temático sapiente,Modi consequuntur delectus iure soluta. Ipsam ...,3266.93,Parque Temático,4
4,1,100004,Mercado ut,Doloribus nisi nulla in nesciunt minima vel. P...,1427.04,Mercado,5
...,...,...,...,...,...,...,...
22440,225,122440,Feria soluta,Eaque amet nostrum ipsam cum. Debitis asperior...,2076.70,Feria,22441
22441,225,122441,Exposición quod,Officia magnam consequatur accusamus. Repellat...,1395.82,Exposición,22442
22442,225,122442,Playa autem,Inventore molestias doloremque officia ab.,657.18,Playa,22443
22443,225,122443,Educativa excepturi,Dolore officiis corporis natus tempore consequ...,2678.69,Educativa,22444


## Separar en recurrente y eventual

In [8]:
df_actividades_recurrentes = df_actividades.sample(frac=0.8, random_state=42)
df_actividades_eventuales = df_actividades.drop(df_actividades_recurrentes.index)

In [9]:
df_actividades_recurrentes['tipo'] = ['recurrente' for _ in range(len(df_actividades_recurrentes))]
df_actividades_eventuales['tipo'] = ['eventual' for _ in range(len(df_actividades_eventuales))]

In [10]:
df_actividades = pd.concat([df_actividades_recurrentes, df_actividades_eventuales])
df_actividades

,destino_id,direccion_id,nombre,descripcion,precio,categoria,id,tipo
134,2,100134,Cultural optio,Totam a similique quas. Laudantium unde velit ...,1097.72,Cultural,135,recurrente
3387,33,103387,Recreativa eius,Molestiae quo modi laborum. Explicabo eligendi...,192.67,Recreativa,3388,recurrente
13406,134,113406,Taller ex,Consequatur fuga magnam dolor velit sint animi...,3110.25,Taller,13407,recurrente
5477,55,105477,Recreativa consequatur,Omnis minus consectetur nesciunt nulla ipsam p...,2756.58,Recreativa,5478,recurrente
20177,202,120177,Gastronomía voluptas,Adipisci sint minima distinctio eligendi. Quia...,4040.76,Gastronomía,20178,recurrente
...,...,...,...,...,...,...,...,...
22427,225,122427,Deportiva sunt,Dolor cumque natus. Fugit quod alias repellendus.,4750.45,Deportiva,22428,eventual
22428,225,122428,Feria esse,Eius facere laudantium officia repudiandae qui...,1955.96,Feria,22429,eventual
22435,225,122435,Recreativa iusto,Molestiae sequi autem explicabo. Sunt dignissi...,1407.61,Recreativa,22436,eventual
22442,225,122442,Playa autem,Inventore molestias doloremque officia ab.,657.18,Playa,22443,eventual


## Exportar

In [11]:
df_actividades.to_csv('./actividades.csv', index=False)
df_direcciones.to_csv('./direcciones_actividades.csv', index=False)

## Visualizar en mapa

In [12]:
# Centrar el mapa (por ejemplo en Sudamérica o Europa)
map_center = [df_direcciones.iloc[0].lat, df_direcciones.iloc[0].lon]  # Ajustá según tus ciudades

# Crear el mapa base
m = folium.Map(location=map_center, zoom_start=12)

# Añadir marcadores al mapa
for _, row in df_direcciones.loc[9900:10000].iterrows():
    folium.Marker(
        location=[row['lat'], row['lon']],
        popup=f"<b>{row['nombre']}</b>",
        tooltip=row['nombre'],
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(m)

In [13]:
m